In [ ]:
!pip install pandas xlsxwriter openpyxl faker

In [16]:
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font

In [ ]:
# устанавливаем входной и выходной файлы
#output_file="output.xlsx"
# Без заголовков!!!!!
# читаем
#df = pd.read_excel(input_file, header=None, names=['group', 'subgroup'])
#print("Первые 5 строк исходного файлика:***")
#display(df.head())

In [17]:
# Задаем переменные для работы основных функций
input_file = "Входной_файлик.xlsx"
groups_number={}
### Функция для разборки и корректировки
filename = input_file
sheet_name = "Sheet"
column_letter_group = "A"
column_letter_subgroup = "B"
### Функция для проверки и подсветки изменений
column="C",
start_row=2

In [18]:
def write_and_highlight_diff(filename, sheet_name, data_dict, column="C", start_row=2):
    """
    Записывает значения словаря в столбец и подсвечивает различия с столбцом B

    :param filename: путь к файлу Excel
    :param sheet_name: название листа
    :param data_dict: словарь со списками значений
    :param column: целевой столбец (по умолчанию 'C')
    :param start_row: начальная строка (по умолчанию 2)
    """
    # Загружаем или создаем файл
    try:
        wb = load_workbook(filename)
        if sheet_name not in wb.sheetnames:
            wb.create_sheet(sheet_name)
    except FileNotFoundError:
        wb = Workbook()
        wb.active.title = sheet_name

    sheet = wb[sheet_name]
    current_row = start_row

    # 1. Записываем данные в столбец C
    for values_list in data_dict.values():
        for value in values_list:
            sheet[f"{column}{current_row}"] = value
            current_row += 1

    # 2. Проверяем различия между столбцами B и C
    # Создаем стиль для заливки (желтый цвет)
    yellow_fill = PatternFill(
        start_color="FFFF00",
        end_color="FFFF00",
        fill_type="solid"
    )

    for row in range(start_row, current_row):
        cell_b = sheet[f"B{row}"]
        cell_c = sheet[f"C{row}"]

        # Сравниваем значения с учетом разных типов
        val_b = cell_b.value
        val_c = cell_c.value

        # Проверяем различия (с обработкой None, NaN и разных типов)
        if isinstance(val_b, float) and isinstance(val_c, float):
            # Для float используем точное сравнение с учетом NaN
            import math
            if math.isnan(val_b) and math.isnan(val_c):
                continue  # Оба NaN - считаем равными
            if math.isnan(val_b) or math.isnan(val_c):
                is_diff = True
            else:
                is_diff = abs(val_b - val_c) > 1e-9
        else:
            # Для остальных типов используем обычное сравнение
            is_diff = val_b != val_c

        # Закрашиваем если значения разные
        if is_diff:
            cell_c.fill = yellow_fill

    # Сохраняем изменения
    wb.save(filename)
    print(f"Данные записаны в столбец {column}, строки {start_row}-{current_row-1}")
    print(f"Проверка различий завершена. Ячейки с отличиями закрашены.")

In [30]:
from collections import defaultdict
def iterate_excel_column(filename, sheet_name, column_letter_group, column_letter_subgroup):
    try:
        workbook = load_workbook(filename)
        sheet = workbook[sheet_name]
        #sheet = workbook.active
        column_index_group = ord(column_letter_group.upper()) - ord('A') + 1  # Преобразуем букву в индекс
        column_index_subgroup = ord(column_letter_subgroup.upper()) - ord('A') + 1  # Преобразуем букву в индекс
        #print(column_index_group)
        #print(column_index_subgroup)
        # посчитаем количество групп и создадим словарь, где количество ключей = количеству групп. Значения будут = значениям подгрупп
        #groups_number={}
        tecNumberGroup = 0
        for row in sheet.iter_rows(min_row=2, values_only=True):
            cell_value_group=row[column_index_group - 1]
            if tecNumberGroup<cell_value_group:
                tecNumberGroup=cell_value_group
                groups_number[cell_value_group]=[]
        print(f"Количество групп = {len(groups_number)}")
        #print(f" Dict {groups_number}")

        # считываем подгруппы
        stoppper=0
        for row in sheet.iter_rows(min_row=2, values_only=True): #начинаем со второй строки
            cell_value_group=row[column_index_group - 1]
            cell_value_subgroup=row[column_index_subgroup - 1]
            if stoppper<cell_value_group:
                stoppper=cell_value_group
            #print(f"Stopper = {stoppper} ***** cell_value_group = {cell_value_group} ***** cell_value_subgroup = {cell_value_subgroup}")
            # начинаем собирать наш словарик подгруппами
            if cell_value_subgroup is None: cell_value_subgroup=0
            groups_number[stoppper].append(cell_value_subgroup)
        #print(f"Dict ALL == {groups_number}")
        #print(f"stopper == {stoppper}")
        #print(f" len dir ===--- {len(groups_number)}")
        #print(f"list in dict {len(groups_number[3])}")
        #print(f"max in 3 key {max(groups_number[2])}")

        for numberKEY in range (1, len(groups_number)+1):
          maxLenListSUB = len(groups_number[numberKEY])
          print(f"Исходный для группы № [{numberKEY}] список подгрупп \n{groups_number[numberKEY]}")
          # Перебиваем значения подгрупп от 1 до максимального значения
          groups_number[numberKEY] = list(range(1, len(groups_number[numberKEY])+1))
          print(f"Скорректированный для группы № [{numberKEY}] список подгрупп \n{groups_number[numberKEY]}")
        # Запускаем функцию-сравнилку
        write_and_highlight_diff(
        filename=input_file,
        sheet_name="Sheet",
        data_dict=groups_number,
        column="C",
        start_row=2
    )

    except FileNotFoundError:
        print(f"Ошибка: Файл '{filename}' не найден.")
    except KeyError:
        print(f"Ошибка: Лист '{sheet_name}' не найден.")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

In [31]:
#!!!!!!!!!!! ***** Run def for work *********** !!!!!!!!!!!!
iterate_excel_column(filename, sheet_name, column_letter_group, column_letter_subgroup)

Количество групп = 3
Исходный для группы № [1] список подгрупп 
[1, 1, 2, 3, 3, 3, 0, 0, 4, 4, 5, 6, 7, 8, 9, 11, 12, 13, 0, 2, 3, 2, 2, 5, 5, 1, 0]
Скорректированный для группы № [1] список подгрупп 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
Исходный для группы № [2] список подгрупп 
[1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 3, 4, 5, 6, 9, 8, 10, 11, 12, 13, 15, 8, 8, 15, 15]
Скорректированный для группы № [2] список подгрупп 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Исходный для группы № [3] список подгрупп 
[1, 2, 2, 3, 4, 0, 0, 5, 6, 7]
Скорректированный для группы № [3] список подгрупп 
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Данные записаны в столбец C, строки 2-63
Проверка различий завершена. Ячейки с отличиями закрашены.
